# Basic Plotting for Cactus

In [ ]:
%matplotlib notebook

In [ ]:
import os, sys, re, h5py
import numpy as np
print("Cactus files to visualize:")
for file in os.listdir("output"):
    if re.match(r'.*\.h5',file):
        print(" ",file)

In [ ]:
import matplotlib.pyplot as plt
print("Color Schemes:")
for cm in dir(plt.cm):
    if hasattr(getattr(plt.cm,cm),'is_gray'):
        print(cm,end=' ')
print()

# Set the color scheme you wish to use below...
global_cmap = plt.cm.rainbow 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def loadh5(fname,tm):
    f = h5py.File("output/"+fname)
    data = {}
    for nm in f:
        if not hasattr(f[nm],"shape"):
            continue
        g = re.search(r'it=(\d+)\s+tl=\d+\s+(rl=\d+\s+c=\d+)',nm)
        if g:
            it = int(g.group(1))
            if it != tm:
                continue
            data[g.group(2)] = f[nm]
    return data

# Your basic surface plot
def assemble_data(fname,tm):
    data = loadh5(fname,tm)
    xvals = loadh5("x.xy.h5",0)
    yvals = loadh5("y.xy.h5",0)
    xs = {}
    ys = {}
    for nm in data:
        for x in np.unique(xvals[nm]):
            xs[x] = 0
        for y in np.unique(yvals[nm]):
            ys[y] = 0
    xn = sorted(xs.keys())
    yn = sorted(ys.keys())
    for i in range(len(xn)):
        xs[xn[i]] = i
    for j in range(len(yn)):
        ys[yn[j]] = j
    ndata = np.zeros((len(yn),len(xn)))
    nxvals = np.zeros((len(yn),len(xn)))
    nyvals = np.zeros((len(yn),len(xn)))
    for nm in data:
        xl1, xh1 = xs[xvals[nm][0,0]], xs[xvals[nm][-1,-1]]+1
        yl1, yh1 = ys[yvals[nm][0,0]], ys[yvals[nm][-1,-1]]+1
        ndata[yl1:yh1, xl1:xh1] = data[nm]
        nxvals[yl1:yh1, xl1:xh1] = xvals[nm]
        nyvals[yl1:yh1, xl1:xh1] = yvals[nm]
    return (nxvals, nyvals, ndata)

def colorplot(fname,tm):
    xvals, yvals, data = assemble_data(fname,tm)
    ax = plt.gca()
    ax.pcolormesh(xvals,yvals,data,cmap=global_cmap)
    plt.show()

def surfaceplot(fname,tm):
    xvals, yvals, data = assemble_data(fname,tm)
    ax = plt.gca(projection='3d')
    ax.plot_surface(xvals,yvals,data,cmap=global_cmap)
    plt.show()

def vectorplot(tm,scale):
    x, y, u = assemble_data("u.xy.h5",tm)
    x, y, v = assemble_data("v.xy.h5",tm)
    c = np.abs(np.arctan2(u,v))
    ax = plt.gca()
    ax.quiver(x,y,u,v,c,scale=scale)
    plt.show()

#plt.figure(figsize=(18, 16), dpi= 80)
colorplot("eta.xy.h5",0)
surfaceplot("eta.xy.h5",0)
vectorplot(1,2)